In [1]:
import sys
sys.path.append("./defect_atom_diag")

In [2]:
import numpy as np
import pandas as pd
from functools import lru_cache as cache
from apply_sym_wan import make_ylm_wanns

In [3]:
N_MESH: int = 200

In [4]:
@cache
def ylm_wrapper(ll, rad_scale):
    return make_ylm_wanns(ll, (N_MESH,)*3, "spherical", rad_scale)

In [5]:
def integrate(A, delr):
    """
    Take the numerical integration (using summation approximation)
    
    Parameters:
        A (3d array): matrix of values to be integrated
        delr (list[float]): (dx, dy, dz)
    """
    return np.sum(A) * np.product(delr)


def avg_ang_mom(psi, h_bar=1):
    """
    Take the average of angular momentum using wave function.
    
    Parameters:
        psi (3d array): wave function
        delr (list[float]): (dx, dy, dz)
        h_bar (float): value of reduced planck constant. Uses 1 by default.
    """
    # check if psi is real:
#     if np.all(np.isreal(psi)):
#         raise ValueError("psi is a real matrix")
    
    # construct axes:
    x = np.linspace(-1, 1, psi.shape[0])
    y = np.linspace(-1, 1, psi.shape[1])
    z = np.linspace(-1, 1, psi.shape[2])
    delr = (x[1]-x[0], y[1]-y[0], z[1]-z[0])
    xx, yy, zz = np.meshgrid(x, y, z, indexing="ij")
    
    if psi.shape != xx.shape:
        raise ValueError("Dimension mismatch")
        
    # complex conjugate of wave function:
    psi_conj = np.conjugate(psi)
    
    # partial derivatives:
    dpsi_dx, dpsi_dy, dpsi_dz = np.gradient(psi, *delr)
    
    # value of operator in each dimension:
    const = -1j * h_bar  # constant (-i*h_bar)
    L_x_op = const * (yy * dpsi_dz - zz * dpsi_dy)
    L_y_op = const * (zz * dpsi_dx - xx * dpsi_dz)
    L_z_op = const * (xx * dpsi_dy - yy * dpsi_dx)
    
    # average in each dimension
    L_x = np.real(integrate(psi_conj * L_x_op, delr))
    L_y = np.real(integrate(psi_conj * L_y_op, delr))
    L_z = np.real(integrate(psi_conj * L_z_op, delr))
    
    return L_x, L_y, L_z

In [6]:
def create_table(ll_range=[0, 3], rad_scale=0.01, h_bar=1):
    min_l = ll_range[0]
    max_l = ll_range[-1]

    col_headers = [f"m={m}" for m in range(-max_l, max_l + 1)]
    row_headers = [f"l={l}" for l in range(min_l, max_l + 1)]

    data = []
    
    for l in range(min_l, max_l + 1):
        psi = ylm_wrapper(l, rad_scale)[0]
        
        row = []
        
        i = 0
        for m in range(-max_l, max_l + 1):
            if -l <= m <= l:
#                 Lx, Ly, Lz = 
#                 print((Lx, Ly, Lz))
                # Store the vector as a tuple
                row.append(tuple("{:.2e}".format(value) \
                                 for value in avg_ang_mom(psi[i], h_bar)))
                i += 1
            else:
                row.append("-")
            
        data.append(row)

    df = pd.DataFrame(data, index=row_headers, columns=col_headers)
    return df

In [7]:
df = create_table(ll_range=[0, 5], h_bar=1)

In [8]:
df

,m=-5,m=-4,m=-3,m=-2,m=-1,m=0,m=1,m=2,m=3,m=4,m=5
l=0,-,-,-,-,-,"(0.00e+00, 0.00e+00, 0.00e+00)",-,-,-,-,-
l=1,-,-,-,-,"(-1.93e-02, -2.06e-03, -8.52e+16)","(0.00e+00, 0.00e+00, 0.00e+00)","(1.93e-02, 2.06e-03, 8.52e+16)",-,-,-,-
l=2,-,-,-,"(1.26e-19, 9.92e-20, -1.08e+01)","(4.00e-02, -7.83e-03, -1.58e+17)","(0.00e+00, 0.00e+00, 0.00e+00)","(-6.20e-02, -3.15e-03, 2.27e+17)","(-1.15e-18, -6.35e-19, 1.08e+01)",-,-,-
l=3,-,-,"(-2.61e-02, 1.04e-02, -2.50e+17)","(1.29e-18, 1.10e-18, -1.42e+01)","(-2.31e-02, -9.70e-03, -1.19e+17)","(0.00e+00, 0.00e+00, 0.00e+00)","(2.62e-02, 1.09e-02, 7.83e+16)","(-4.33e-19, -2.91e-18, 1.42e+01)","(1.30e-02, -6.14e-03, 2.50e+17)",-,-
l=4,-,"(-9.28e-18, -2.59e-18, -5.20e+01)","(-4.35e-02, 8.64e-03, -2.89e+17)","(3.03e-18, -2.89e-20, -2.13e+01)","(-7.76e-02, -3.39e-03, -1.41e+17)","(0.00e+00, 0.00e+00, 0.00e+00)","(9.41e-02, 1.66e-01, 2.03e+17)","(-3.66e-18, -1.71e-18, 2.13e+01)","(-1.79e-02, -2.47e-03, 2.89e+17)","(1.02e-17, -1.31e-18, 5.20e+01)",-
l=5,"(-2.35e-02, -7.36e-03, -1.55e+17)","(7.56e-18, -3.90e-18, -6.31e+01)","(-7.70e-02, 1.52e-02, -3.03e+17)","(-4.90e-18, -6.93e-18, -2.27e+01)","(1.14e-01, -3.89e-02, -1.40e+17)","(0.00e+00, 0.00e+00, 0.00e+00)","(3.62e-02, 9.34e-03, 1.35e+17)","(-3.44e-18, -3.82e-18, 2.27e+01)","(-3.16e-02, 4.63e-02, 2.84e+17)","(-8.14e-18, 6.33e-18, 6.31e+01)","(1.99e-02, 1.13e-02, 1.53e+17)"
